In [1]:
import sys
import os
from accelerate import notebook_launcher
from datasets import load_dataset
import logging


# Configure logging:
# (i) lear any pre-existing logging handlers (especially useful in notebook reruns)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
# (ii) now safely configure logging
logging.getLogger("codecarbon").setLevel(logging.ERROR)
logging.basicConfig(level=logging.INFO)

# Adjust paths -> import classes
project_root = os.getcwd()  
if project_root not in sys.path:
    sys.path.append(project_root)
from classes.experiment_config import ExperimentConfig
from classes.experiment_runner import ExperimentRunner

#import torch.multiprocessing as mp
#mp.set_start_method('spawn', force=True)

In [2]:
# Load prompts (here, using the 'arxiv' split from the lighteval/pile_helm dataset)
ds = load_dataset("lighteval/pile_helm", "arxiv")["test"]
prompts = [sample["text"] for sample in ds]

In [ ]:
experiment_config = ExperimentConfig(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    is_encoder_decoder="decoder_only",
    task_type="text_generation",
    inference_type="pure_generative",  
    max_input_tokens=512,
    max_output_tokens=50,
    num_input_prompts=50,
    save_outputs=True,
    decode_token_to_text=True,
    gpu_list=[0,2,3],
    num_processes=2,
    batching_options={
        "fixed_max_batch_size": 2, # this is max batch size if adaptive batching on; fixed batch size if it's off
        "adaptive_batching": False,
        "adaptive_max_tokens": 512  
    },
    sharding_config={
        "fsdp_config": {
            "use_orig_params": True,
            "cpu_offload": True
        },
        "sharding_strategy": "NO_SHARD"
    },
    query_rate=1,
    decoder_temperature=1.0, # NB: needs to be a float
    fp_precision="float32",
    backend="pytorch"
)


# function to run the whole experiment workflow.
def run_experiment_workflow():
    
    # clean up processes: (IS THIS BAD PRACTICE TO HAVE HERE?)
    # cleanup_port(29500)
    
    # run experiment
    runner = ExperimentRunner(experiment_config, prompts)
    runner.run_torch()
    
    # If not main process, exit cleanly after run
    #if not runner.accelerator.is_main_process:
    #    return  # do NOT proceed further

    # Only main process continues
    if runner.accelerator.is_main_process:
         # aggregate results
        runner.aggregate_results()
        # save results
        runner.save_experiment_results()  
        
    # could add in script here to delete JSON files if storage an issue
    
    # clean up processes: 
    # KEEPS CAUSING CRASHES -> accelerator.wait_for_everyone()?
    # runner.teardown()

    return

# Launch the experiment across the specified number of processes.
notebook_launcher(run_experiment_workflow,
                    num_processes=experiment_config.num_processes) 
                    #terminate_on_error=True) # USE THIS IN accelerate.launch.launch()                 

Launching training on 2 GPUs.
Accelerator set up
Unique experiment id: 0163
TinyLlama/TinyLlama-1.1B-Chat-v1.0 loaded using pytorch, with precision float32
Original generate method saved.
Model and tokenizer prepared


INFO:classes.experiment_runner:[Process 2051946] Model is on device: cuda:0
INFO:classes.experiment_runner:[Process 2051947] Model is on device: cuda:1


Original generate method reassigned


INFO:classes.experiment_runner:[Process 2051946] Dummy forward pass complete
INFO:classes.experiment_runner:[Process 2051947] Dummy forward pass complete


Prompts processed: 50 prompts.
Energy tracking started
Task type: pure_generative
Using fixed batching (non-adaptive): created 25 batches.


INFO:classes.experiment_runner:[Process 2051947] Inference complete
INFO:classes.experiment_runner:[Process 2051946] Inference complete


Decoded token outputs successfully.
Energy tracking stopped
Saved text outputs


INFO:classes.experiment_runner:Process 1 saved its energy metrics.
W0324 00:56:30.512000 2051629 torch/distributed/elastic/agent/server/api.py:704] Received Signals.SIGINT death signal, shutting down workers
W0324 00:56:30.515000 2051629 torch/distributed/elastic/multiprocessing/api.py:766] Closing process 2051946 via signal SIGINT
W0324 00:56:30.518000 2051629 torch/distributed/elastic/multiprocessing/api.py:766] Closing process 2051947 via signal SIGINT
